In [1]:
# Import PySpark libraries only
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Create Spark Session
spark = SparkSession.builder \
    .appName("EV_Data_Preprocessing_EDA") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "6g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

print(f"Spark version: {spark.version}")
print(f"Spark UI: {spark.sparkContext.uiWebUrl}")
print("✓ Spark session created")
print("\nUsing PySpark only - no visualization libraries")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/26 14:59:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 3.5.1
Spark UI: http://10.233.81.94:4040
✓ Spark session created

Using PySpark only - no visualization libraries


## 1. Load Dataset from HDFS

In [2]:
# Load dataset from HDFS
hdfs_path = "hdfs://localhost:9000/des/data/ev_ice_timeseries_1667_locations_120months_200krows_tweaked_for_events_and_noise.csv"

print("Loading data from HDFS...")
df = spark.read.csv(
    hdfs_path,
    header=True,
    inferSchema=True,
    mode="PERMISSIVE",
    nullValue="NULL",
    nanValue="NaN"
)

print(f"\n✓ Loaded {df.count():,} rows, {len(df.columns)} columns")
print("\nSchema:")
df.printSchema()

Loading data from HDFS...



✓ Loaded 200,040 rows, 35 columns

Schema:
root
 |-- city: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- fuel_price_per_litre: double (nullable = true)
 |-- avg_city_aqi: double (nullable = true)
 |-- charging_stations_per_10k: double (nullable = true)
 |-- ev_subsidy_amount: double (nullable = true)
 |-- income_level: double (nullable = true)
 |-- vehicle_price_ev: double (nullable = true)
 |-- vehicle_price_ice: double (nullable = true)
 |-- battery_range_km: double (nullable = true)
 |-- charging_time_minutes: double (nullable = true)
 |-- maintenance_cost_ev: double (nullable = true)
 |-- maintenance_cost_ice: double (nullable = true)
 |-- fuel_efficiency_ice_kmpl: double (nullable = true)
 |-- electricity_cost_per_kwh: double (nullable = true)
 |-- co2_emission_city: double (nullable = true)
 |-- population_density: double (nullable = true)
 |-- public_transport_score: double (nullable = 

In [3]:
# Show sample data
print("Sample data (first 20 rows):")
df.show(20)

Sample data (first 20 rows):
+------+----+-----+----------+--------------------+------------+-------------------------+-----------------+------------+----------------+-----------------+----------------+---------------------+-------------------+--------------------+------------------------+------------------------+-----------------+------------------+----------------------+------------------------+------------------+---------------------------+--------------------------+----------------------+-------------------------+----------------------------+-----------------------+------------------------+-----------------------+------------------+-------------------+-----------------------------+-----------------------+-----------+
|  city|year|month|      date|fuel_price_per_litre|avg_city_aqi|charging_stations_per_10k|ev_subsidy_amount|income_level|vehicle_price_ev|vehicle_price_ice|battery_range_km|charging_time_minutes|maintenance_cost_ev|maintenance_cost_ice|fuel_efficiency_ice_kmpl|electric

25/11/26 15:00:24 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
# Column names
print(f"Total columns: {len(df.columns)}")
print("\nColumn names:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

Total columns: 35

Column names:
 1. city
 2. year
 3. month
 4. date
 5. fuel_price_per_litre
 6. avg_city_aqi
 7. charging_stations_per_10k
 8. ev_subsidy_amount
 9. income_level
10. vehicle_price_ev
11. vehicle_price_ice
12. battery_range_km
13. charging_time_minutes
14. maintenance_cost_ev
15. maintenance_cost_ice
16. fuel_efficiency_ice_kmpl
17. electricity_cost_per_kwh
18. co2_emission_city
19. population_density
20. public_transport_score
21. traffic_congestion_index
22. ev_awareness_score
23. gov_infra_investment_crores
24. charging_infra_growth_rate
25. fuel_price_growth_rate
26. aqi_improvement_potential
27. consumer_range_anxiety_score
28. vehicle_resale_value_ev
29. vehicle_resale_value_ice
30. city_ev_readiness_index
31. ev_sales_last_year
32. ice_sales_last_year
33. charging_cost_per_full_charge
34. ev_adoption_probability
35. state


## 2. Data Quality Analysis

In [5]:
# Analyze data quality
print("="*80)
print("DATA QUALITY ANALYSIS")
print("="*80)

total_rows = df.count()
total_cols = len(df.columns)

print(f"\nDataset dimensions: {total_rows:,} rows × {total_cols} columns")
print(f"Total cells: {total_rows * total_cols:,}")

# Count nulls per column
print("\nAnalyzing missing values...")
null_counts = []
for col_name in df.columns:
    null_count = df.filter(F.col(col_name).isNull()).count()
    null_pct = (null_count / total_rows) * 100
    null_counts.append((col_name, null_count, null_pct))

null_df = spark.createDataFrame(null_counts, ['column', 'null_count', 'null_percentage'])

print("\nColumns with missing values (Top 20):")
null_df.filter(F.col('null_count') > 0).orderBy(F.col('null_count').desc()).show(20, truncate=False)

total_nulls = null_df.agg(F.sum('null_count')).collect()[0][0]
null_pct_overall = (total_nulls / (total_rows * total_cols)) * 100
print(f"\nTotal missing cells: {total_nulls:,} ({null_pct_overall:.2f}% of all cells)")

DATA QUALITY ANALYSIS

Dataset dimensions: 200,040 rows × 35 columns
Total cells: 7,001,400

Analyzing missing values...

Columns with missing values (Top 20):
+-------------------------+----------+------------------+
|column                   |null_count|null_percentage   |
+-------------------------+----------+------------------+
|fuel_price_per_litre     |1999      |0.9993001399720055|
|city_ev_readiness_index  |1997      |0.9983003399320136|
|ev_sales_last_year       |1997      |0.9983003399320136|
|ev_subsidy_amount        |1996      |0.9978004399120176|
|avg_city_aqi             |1995      |0.9973005398920216|
|charging_stations_per_10k|1995      |0.9973005398920216|
|ev_awareness_score       |1993      |0.9963007398520296|
|income_level             |1993      |0.9963007398520296|
|charging_time_minutes    |1992      |0.9958008398320336|
|ice_sales_last_year      |1992      |0.9958008398320336|
|battery_range_km         |1992      |0.9958008398320336|
+-------------------------+-

In [6]:
# Check for duplicates
print("\nChecking for duplicate rows...")
distinct_rows = df.distinct().count()
duplicates = total_rows - distinct_rows

print(f"Total rows: {total_rows:,}")
print(f"Distinct rows: {distinct_rows:,}")
print(f"Duplicate rows: {duplicates:,}")

if duplicates > 0:
    print(f"Duplicate percentage: {(duplicates/total_rows)*100:.2f}%")
else:
    print("✓ No duplicates found")


Checking for duplicate rows...


Total rows: 200,040
Distinct rows: 200,040
Duplicate rows: 0
✓ No duplicates found


## 3. Data Cleaning

In [7]:
# Clean the dataset
print("="*80)
print("DATA CLEANING")
print("="*80)

print(f"\nOriginal dataset: {df.count():,} rows")

# Step 1: Replace 'None' strings with nulls
print("\n1. Standardizing null values...")
for column in df.columns:
    df = df.withColumn(
        column,
        F.when(F.col(column).isin(['None', 'NULL', '', 'NaN']), None).otherwise(F.col(column))
    )
print("   ✓ Null values standardized")

# Step 2: Remove duplicates
print("\n2. Removing duplicate rows...")
original_count = df.count()
df = df.dropDuplicates()
duplicates_removed = original_count - df.count()
print(f"   ✓ Removed {duplicates_removed:,} duplicates")

# Step 3: Impute missing numeric values
print("\n3. Imputing missing numeric values with median...")
numeric_cols = [field.name for field in df.schema.fields 
               if str(field.dataType) in ['IntegerType', 'DoubleType', 'FloatType', 'LongType']]

target_col = 'ev_adoption_probability'
imputed_count = 0

for col_name in numeric_cols:
    if col_name != target_col:
        null_count = df.filter(F.col(col_name).isNull()).count()
        if null_count > 0:
            median_val = df.stat.approxQuantile(col_name, [0.5], 0.01)[0]
            df = df.withColumn(col_name, F.when(F.col(col_name).isNull(), median_val).otherwise(F.col(col_name)))
            imputed_count += null_count
            print(f"   - {col_name}: imputed {null_count:,} values with median {median_val:.4f}")

print(f"   ✓ Total numeric values imputed: {imputed_count:,}")

# Step 4: Fill categorical nulls
print("\n4. Filling missing categorical values...")
categorical_cols = [field.name for field in df.schema.fields if str(field.dataType) == 'StringType']

for col_name in categorical_cols:
    null_count = df.filter(F.col(col_name).isNull()).count()
    if null_count > 0:
        df = df.withColumn(col_name, F.when(F.col(col_name).isNull(), 'Unknown').otherwise(F.col(col_name)))
        print(f"   - {col_name}: filled {null_count:,} nulls with 'Unknown'")

print("   ✓ Categorical values filled")

# Step 5: Drop rows with null target
print("\n5. Removing rows with null target variable...")
before_target_drop = df.count()
df = df.filter(F.col(target_col).isNotNull())
target_nulls_removed = before_target_drop - df.count()
print(f"   ✓ Removed {target_nulls_removed:,} rows with null target")

print(f"\n✓ CLEANED DATASET: {df.count():,} rows")
print("="*80)

DATA CLEANING

Original dataset: 200,040 rows

1. Standardizing null values...
   ✓ Null values standardized

2. Removing duplicate rows...


   ✓ Removed 0 duplicates

3. Imputing missing numeric values with median...
   ✓ Total numeric values imputed: 0

4. Filling missing categorical values...
   ✓ Categorical values filled

5. Removing rows with null target variable...


   ✓ Removed 0 rows with null target



✓ CLEANED DATASET: 200,040 rows


## 4. Descriptive Statistics

In [8]:
# Descriptive statistics for numeric columns
print("="*80)
print("DESCRIPTIVE STATISTICS")
print("="*80)

print("\nNumeric columns summary:")
df.describe().show(truncate=False)

DESCRIPTIVE STATISTICS

Numeric columns summary:


+-------+---------+-----------------+------------------+--------------------+------------------+-------------------------+-----------------+------------------+------------------+------------------+------------------+---------------------+-------------------+--------------------+------------------------+------------------------+------------------+------------------+----------------------+------------------------+------------------+---------------------------+--------------------------+----------------------+-------------------------+----------------------------+-----------------------+------------------------+-----------------------+------------------+-------------------+-----------------------------+-----------------------+---------------------------+
|summary|city     |year             |month             |fuel_price_per_litre|avg_city_aqi      |charging_stations_per_10k|ev_subsidy_amount|income_level      |vehicle_price_ev  |vehicle_price_ice |battery_range_km  |charging_time_minutes|

In [9]:
# Target variable statistics
print(f"\nTarget variable: {target_col}")
df.select(target_col).describe().show()

# Percentiles
percentiles = df.stat.approxQuantile(target_col, [0.25, 0.50, 0.75, 0.95, 0.99], 0.01)
print(f"25th percentile: {percentiles[0]:.6f}")
print(f"50th percentile (median): {percentiles[1]:.6f}")
print(f"75th percentile: {percentiles[2]:.6f}")
print(f"95th percentile: {percentiles[3]:.6f}")
print(f"99th percentile: {percentiles[4]:.6f}")


Target variable: ev_adoption_probability


+-------+-----------------------+
|summary|ev_adoption_probability|
+-------+-----------------------+
|  count|                 200040|
|   mean|    0.33426829634073285|
| stddev|    0.11261165851046953|
|    min|                  -0.68|
|    max|                   1.34|
+-------+-----------------------+

25th percentile: 0.260000
50th percentile (median): 0.330000
75th percentile: 0.400000
95th percentile: 0.500000
99th percentile: 1.340000


## 5. Feature Correlations

In [13]:
# Calculate correlations with target
print("="*80)
print("FEATURE CORRELATIONS WITH TARGET")
print("="*80)

correlations = []
for col_name in numeric_cols:
    if col_name != target_col:
        corr = df.stat.corr(col_name, target_col)
        correlations.append((col_name, corr))

corr_df = spark.createDataFrame(correlations, ['feature', 'correlation'])

print("\nTop 15 Positive Correlations:")
corr_df.orderBy(F.col('correlation').desc()).show(15, truncate=False)

print("\nTop 15 Negative Correlations:")
corr_df.orderBy(F.col('correlation').asc()).show(15, truncate=False)

FEATURE CORRELATIONS WITH TARGET


PySparkValueError: [CANNOT_INFER_EMPTY_SCHEMA] Can not infer schema from empty dataset.

## 6. Temporal Analysis

In [14]:
# Extract temporal features
print("="*80)
print("TEMPORAL ANALYSIS")
print("="*80)

if 'date' in df.columns:
    df = df.withColumn('date_parsed', F.to_date(F.col('date')))
    df = df.withColumn('year', F.year('date_parsed'))
    df = df.withColumn('month', F.month('date_parsed'))
    df = df.withColumn('quarter', F.quarter('date_parsed'))
    
    # Yearly trends
    print("\nYearly Average EV Adoption Probability:")
    yearly_stats = df.groupBy('year').agg(
        F.count('*').alias('count'),
        F.avg(target_col).alias('avg_adoption'),
        F.stddev(target_col).alias('std_adoption'),
        F.min(target_col).alias('min_adoption'),
        F.max(target_col).alias('max_adoption')
    ).orderBy('year')
    yearly_stats.show(truncate=False)
    
    # Monthly patterns
    print("\nMonthly Average EV Adoption Probability:")
    monthly_stats = df.groupBy('month').agg(
        F.count('*').alias('count'),
        F.avg(target_col).alias('avg_adoption'),
        F.stddev(target_col).alias('std_adoption')
    ).orderBy('month')
    monthly_stats.show(12, truncate=False)
    
    # Quarterly patterns
    print("\nQuarterly Average EV Adoption Probability:")
    quarterly_stats = df.groupBy('quarter').agg(
        F.count('*').alias('count'),
        F.avg(target_col).alias('avg_adoption')
    ).orderBy('quarter')
    quarterly_stats.show(truncate=False)
else:
    print("No 'date' column found in dataset")

TEMPORAL ANALYSIS

Yearly Average EV Adoption Probability:


+----+-----+-------------------+-------------------+------------+------------+
|year|count|avg_adoption       |std_adoption       |min_adoption|max_adoption|
+----+-----+-------------------+-------------------+------------+------------+
|2015|20004|0.25924565086982565|0.09093907496933229|-0.68       |1.34        |
|2016|20004|0.2754374125174963 |0.09229147292479084|-0.68       |1.34        |
|2017|20004|0.29201909618076394|0.09441408969117435|-0.68       |1.34        |
|2018|20004|0.30871675664867054|0.09957525130552115|-0.68       |1.34        |
|2019|20004|0.3324170165966804 |0.10252772830857375|-0.68       |1.34        |
|2020|20004|0.34807438512297534|0.10514367782610054|-0.68       |1.34        |
|2021|20004|0.3613292341531694 |0.10752136372202908|-0.68       |1.34        |
|2022|20004|0.37514597080583795|0.10879663681398906|-0.68       |1.34        |
|2023|20004|0.3888047390521887 |0.11028282604412552|-0.68       |1.34        |
|2024|20004|0.401492701459708  |0.11160919722310048|

## 7. Location-Based Analysis

In [15]:
# Location analysis
print("="*80)
print("LOCATION-BASED ANALYSIS")
print("="*80)

location_col = next((c for c in df.columns if c in ['location', 'city']), None)

if location_col:
    total_locations = df.select(location_col).distinct().count()
    print(f"\nTotal unique locations: {total_locations:,}")
    
    # Location statistics
    location_stats = df.groupBy(location_col).agg(
        F.count('*').alias('count'),
        F.avg(target_col).alias('avg_adoption'),
        F.stddev(target_col).alias('std_adoption'),
        F.min(target_col).alias('min_adoption'),
        F.max(target_col).alias('max_adoption')
    )
    
    print("\nTop 20 Locations by Record Count:")
    location_stats.orderBy(F.col('count').desc()).show(20, truncate=False)
    
    print("\nTop 20 Locations by Average EV Adoption:")
    location_stats.orderBy(F.col('avg_adoption').desc()).show(20, truncate=False)
    
    print("\nBottom 20 Locations by Average EV Adoption:")
    location_stats.orderBy(F.col('avg_adoption').asc()).show(20, truncate=False)
else:
    print("No location column found")

LOCATION-BASED ANALYSIS

Total unique locations: 1,667

Top 20 Locations by Record Count:


+------------------------+-----+-------------------+--------------------+------------+------------+
|city                    |count|avg_adoption       |std_adoption        |min_adoption|max_adoption|
+------------------------+-----+-------------------+--------------------+------------+------------+
|Vasai-Virar             |120  |0.14808333333333332|0.08566394997272757 |-0.68       |0.22        |
|Jharkhand Taluk 5       |120  |0.3572499999999999 |0.10522454980826929 |0.24        |1.34        |
|Madhya Pradesh Taluk 110|120  |0.21791666666666665|0.04373133855702104 |0.14        |0.29        |
|Maharashtra Taluk 70    |120  |0.19133333333333333|0.03645956280439954 |0.13        |0.25        |
|Meghalaya Taluk 5       |120  |0.15866666666666665|0.02950383348780613 |0.11        |0.21        |
|Rajasthan Taluk 34      |120  |0.44883333333333336|0.09024880020463072 |0.37        |1.34        |
|Rajasthan Taluk 39      |120  |0.30191666666666667|0.04457635943780392 |0.23        |0.39        |


In [16]:
# State-level analysis (if applicable)
if 'state' in df.columns:
    print("\nSTATE-LEVEL ANALYSIS")
    print("="*80)
    
    total_states = df.select('state').distinct().count()
    print(f"\nTotal unique states: {total_states}")
    
    state_stats = df.groupBy('state').agg(
        F.count('*').alias('count'),
        F.countDistinct(location_col).alias('num_locations') if location_col else F.lit(0).alias('num_locations'),
        F.avg(target_col).alias('avg_adoption'),
        F.stddev(target_col).alias('std_adoption')
    )
    
    print("\nState Statistics (ordered by record count):")
    state_stats.orderBy(F.col('count').desc()).show(50, truncate=False)
    
    print("\nState Statistics (ordered by average adoption):")
    state_stats.orderBy(F.col('avg_adoption').desc()).show(50, truncate=False)


STATE-LEVEL ANALYSIS

Total unique states: 34

State Statistics (ordered by record count):
+----------------------------------------+-----+-------------+-------------------+-------------------+
|state                                   |count|num_locations|avg_adoption       |std_adoption       |
+----------------------------------------+-----+-------------+-------------------+-------------------+
|Tamil Nadu                              |22200|185          |0.33891036036036043|0.11317334026589393|
|Maharashtra                             |22200|185          |0.3349941441441441 |0.12002514850500794|
|Uttar Pradesh                           |21600|180          |0.3294976851851852 |0.11195096125957925|
|Madhya Pradesh                          |16800|140          |0.3296494047619048 |0.10624553859219285|
|Karnataka                               |12600|105          |0.3419976190476192 |0.12053944545925886|
|Gujarat                                 |12000|100          |0.3326266666666666 |0.

## 8. Save Cleaned Data and Results

In [18]:
# Save cleaned dataset
print("="*80)
print("SAVING OUTPUTS")
print("="*80)

# Save cleaned data to HDFS
output_path = "hdfs://localhost:9000/des/data/ev_ice_cleaned"
print(f"\nSaving cleaned dataset to HDFS: {output_path}")
df.write.csv(output_path, header=True, mode='overwrite')
print("✓ Cleaned dataset saved")

# Save correlation results
corr_output = "outputs/eda/correlations_full"
print(f"\nSaving correlations to: {corr_output}")
corr_df.coalesce(1).write.csv(corr_output, header=True, mode='overwrite')
print("✓ Correlations saved")

# Save location statistics
if location_col:
    location_output = "outputs/eda/location_statistics"
    print(f"\nSaving location statistics to: {location_output}")
    location_stats.coalesce(1).write.csv(location_output, header=True, mode='overwrite')
    print("✓ Location statistics saved")

SAVING OUTPUTS

Saving cleaned dataset to HDFS: hdfs://localhost:9000/des/data/ev_ice_cleaned


✓ Cleaned dataset saved

Saving correlations to: outputs/eda/correlations_full


NameError: name 'corr_df' is not defined

## 9. Summary Report

In [19]:
# Final summary
print("="*80)
print("FINAL SUMMARY")
print("="*80)

print(f"\nOriginal dataset: {total_rows:,} rows")
print(f"Cleaned dataset: {df.count():,} rows")
print(f"Rows removed: {total_rows - df.count():,}")

print(f"\nTotal columns: {len(df.columns)}")
print(f"Numeric features: {len(numeric_cols)}")
print(f"Categorical features: {len(categorical_cols)}")

if location_col:
    print(f"\nTotal locations: {df.select(location_col).distinct().count():,}")

if 'state' in df.columns:
    print(f"Total states: {df.select('state').distinct().count()}")

if 'year' in df.columns:
    years = df.select('year').distinct().orderBy('year').collect()
    print(f"\nYear range: {years[0][0]} - {years[-1][0]}")

print(f"\nTarget variable: {target_col}")
target_stats = df.select(target_col).summary().collect()
for row in target_stats:
    print(f"  {row[0]}: {row[1]}")

print("\n✓ Data preprocessing and EDA complete!")
print("="*80)

FINAL SUMMARY

Original dataset: 200,040 rows


Cleaned dataset: 200,040 rows


Rows removed: 0

Total columns: 37
Numeric features: 0
Categorical features: 0

Total locations: 1,667
Total states: 34

Year range: 2015 - 2024

Target variable: ev_adoption_probability


  count: 200040
  mean: 0.33426829634073285
  stddev: 0.11261165851046953
  min: -0.68
  25%: 0.26
  50%: 0.33
  75%: 0.4
  max: 1.34

✓ Data preprocessing and EDA complete!


In [20]:
# Stop Spark session
spark.stop()
print("✓ Spark session stopped")

✓ Spark session stopped
